In [1]:
import pandas as pd
import numpy as np 
import sys
sys.path.append("../tax-calculator")
from taxcalc.records import Records
from taxcalc import Policy, Records, Calculator, Behavior, behavior
from taxcalc.utils import *
from numpy.testing import assert_array_almost_equal
import matplotlib
import matplotlib.pyplot as plt 
from matplotlib.transforms import BlendedGenericTransform
%matplotlib inline
import copy
import itertools

In [2]:
puf = pd.read_csv("../tax-calculator/puf.csv")
policy_base = Policy()
records_base = Records(puf)

policy_reform = Policy()
records_reform = Records(puf)

policy_org = Policy()
records_org = Records(puf)

calcbase = Calculator(policy = policy_base, records = records_base)
calcreform = Calculator(policy = policy_reform, records = records_reform)
calcori = Calculator(policy = policy_org, records = records_org)

You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.


In [3]:
reform_5mm = {
    2017: {'_II_rt8': [0.436],
           '_II_brk7': [[5000000, 5000000, 5000000, 5000000, 5000000, 5000000]],
           '_CG_rt4':[0.24],
           '_CG_thd3': [[5000000, 5000000, 5000000, 5000000, 5000000, 5000000]],
           '_AMT_CG_rt4':[0.24],
           '_AMT_CG_thd3': [[5000000, 5000000, 5000000, 5000000, 5000000, 5000000]]
        }}
policy_reform.implement_reform(reform_5mm)
calcreform.advance_to_year(2017)
calcbase.advance_to_year(2017)

In [4]:
calcbase.calc_all()
calcreform.calc_all()

In [5]:
EPSILON = 1e-3

In [6]:
source1 = get_mtr_data(calcbase,calcreform, MARS = 1,weights = wage_weighted, income_measure = 'e00200')

In [6]:
def get_mtr_data(calcX, calcY, MARS='ALL', weights='weighted_mean',
                 tab='e00200', mtr_measure='IIT', complex_weight=False):
    df_x = results(calcX)
    df_y = results(calcY)

    a, mtr_iit_x, mtr_combined_x = calcX.mtr()
    a, mtr_iit_y, mtr_combined_y = calcY.mtr()
    df_x['mtr_iit'] = mtr_iit_x
    df_y['mtr_iit'] = mtr_iit_y
    df_x['mtr_combined'] = mtr_combined_x
    df_y['mtr_combined'] = mtr_combined_y

    df_y[tab] = df_x[tab]

    if complex_weight:
        df_x = add_weighted_decile_bins(df_x, tab, 100, complex_weight=True)
        df_y = add_weighted_decile_bins(df_y, tab, 100, complex_weight=True)
    else:
        df_x = add_weighted_decile_bins(df_x, tab, 100)
        df_y = add_weighted_decile_bins(df_y, tab, 100)

    if MARS == 'ALL':
        df_filtered_x = df_x.copy()
        df_filtered_y = df_y.copy()
    else:
        df_filtered_x = df_x[(df_x['MARS'] == MARS)].copy()
        df_filtered_y = df_y[(df_y['MARS'] == MARS)].copy()

    gp_x = df_filtered_x.groupby('bins', as_index=False)
    gp_y = df_filtered_y.groupby('bins', as_index=False)

    if mtr_measure == 'combined':
        wgtpct_x = gp_x.apply(weights, 'mtr_combined')
        wgtpct_y = gp_y.apply(weights, 'mtr_combined')
    elif mtr_measure == 'IIT':
        wgtpct_x = gp_x.apply(weights, 'mtr_iit')
        wgtpct_y = gp_y.apply(weights, 'mtr_iit')

    wpct_x = DataFrame(data=wgtpct_x, columns=['w_mtr'])
    wpct_y = DataFrame(data=wgtpct_y, columns=['w_mtr'])

    wpct_x['bins'] = np.arange(1, 101)
    wpct_y['bins'] = np.arange(1, 101)

    rsltx = pd.merge(df_filtered_x[['bins']], wpct_x, how='left')
    rslty = pd.merge(df_filtered_y[['bins']], wpct_y, how='left')

    df_filtered_x['w_mtr'] = rsltx['w_mtr'].values
    df_filtered_y['w_mtr'] = rslty['w_mtr'].values

    df_filtered_x.drop_duplicates(subset='bins', inplace=True)
    df_filtered_y.drop_duplicates(subset='bins', inplace=True)

    df_filtered_x = df_filtered_x['w_mtr']
    df_filtered_y = df_filtered_y['w_mtr']

    merged = pd.concat([df_filtered_x, df_filtered_y], axis=1,
                       ignore_index=True)
    merged.columns = ['base', 'reform']

    return merged

In [8]:
source2 = get_mtr_data(calcbase,calcreform,weights = wage_weighted, income_measure = 'e00200', complex_weight = True)

In [6]:
from bokeh.models import Plot, Range1d, ImageURL, DataRange1d
from bokeh.embed import components
from bokeh.layouts import layout
from bokeh.plotting import figure, hplot, vplot, output_file, show
from bokeh.models import (ColumnDataSource, LogAxis, LinearAxis, Rect, FactorRange,
                          CategoricalAxis, Line, Text, Square, HoverTool)

from styles import (PLOT_FORMATS,
                    AXIS_FORMATS,
                    FONT_PROPS_SM,
                    DARK_GRAY,
                    GREEN,
                    PURPLE,
                    RED,
                    BLUE)

In [10]:
plot_width = 425
plot_height = 250
text_plot_height = 50

In [9]:
show(mtr_plot(source2))

In [11]:
QQ = figure(plot_width=plot_width,
             plot_height=plot_height,
             title='Effect of Surtax Proposal on Marginal Tax Rate in 2017')

QQ.line( (source2.reset_index()).index,(source2.reset_index()).base,line_color=BLUE,
                     line_width=0.8,
                     line_alpha=.8, legend="Base")

QQ.line( (source2.reset_index()).index,(source2.reset_index()).reform,line_color=RED,
                     line_width=0.8,
                     line_alpha=1, legend="Reform")

QQ.legend.label_text_font = "times"
QQ.legend.label_text_font_style = "italic"
QQ.legend.location = "top_left"

QQ.legend.label_width = 2
QQ.legend.label_height = 2
QQ.legend.label_standoff = 2
QQ.legend.glyph_width = 14
QQ.legend.glyph_height = 14
QQ.legend.legend_spacing = 5
QQ.legend.legend_padding = 5
QQ.yaxis.axis_label = 'Avg. MTR on Wage Income'
QQ.xaxis.axis_label = 'Percentile of Economic Activity'

In [12]:
QQ

In [14]:
show(PP)

In [15]:
a, mtr_base, b = calcbase.mtr()

In [16]:
(calcbase.records.s006 * mtr_base * calcbase.records.e00200).sum()/(calcbase.records.s006 * calcbase.records.e00200).sum()

0.23746676486594379

In [17]:
a, mtr_rfm, b = calcreform.mtr()

In [18]:
(calcreform.records.s006 * mtr_rfm * calcreform.records.e00200).sum()/(calcreform.records.s006 * calcreform.records.e00200).sum()

0.23830699688009577

In [41]:
II = [calcbase.policy.II_brk1, calcbase.policy.II_brk2, calcbase.policy.II_brk3, calcbase.policy.II_brk4, calcbase.policy.II_brk5, calcbase.policy.II_brk6, calcbase.policy.II_brk7]

In [35]:
calcbase.policy._II_brk1[2,3]

13150.0

In [42]:
II[6]

array([  9.12150000e+99,   9.12150000e+99,   9.12150000e+99,
         9.12150000e+99,   9.12150000e+99,   9.12150000e+99])

In [3]:
calcreform.advance_to_year(2013)
calcbase.advance_to_year(2014)
calcbase.calc_all()
calcreform.calc_all()

In [14]:
agi_ratio_5mm = (calcbase.records.c00100[calcbase.records.c00100 >= 5000000]*calcbase.records.s006[calcbase.records.c00100 >= 5000000]).sum()/ (calcbase.records.c00100 *calcbase.records.s006).sum()
agi_ratio_5mm

0.057690758617813233

In [15]:
taxable_5mm = (calcbase.records.c04800[calcbase.records.c00100 >= 5000000]*calcbase.records.s006[calcbase.records.c00100 >= 5000000]).sum()/ (calcbase.records.c04800 *calcbase.records.s006).sum()
taxable_5mm

0.076393693262004403

In [5]:
agi_ratio_5mm = (calcreform.records.c00100[calcreform.records.c00100 >= 5000000]*calcreform.records.s006[calcreform.records.c00100 >= 5000000]).sum()/ (calcreform.records.c00100 *calcreform.records.s006).sum()
agi_ratio_5mm

0.049192354585735024

In [6]:
taxable_5mm = (calcreform.records.c04800[calcreform.records.c00100 >= 5000000]*calcreform.records.s006[calcreform.records.c00100 >= 5000000]).sum()/ (calcreform.records.c04800 *calcreform.records.s006).sum()
taxable_5mm

0.06536916725892182